In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install EMD-signal

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 9.9 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0


In [2]:
import matplotlib.pyplot as plt
import time
import scipy
import scipy.io
from scipy.fft import fft
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from torch.utils import data
from torch.autograd import Variable
import pandas as pd

In [3]:
import sys    
sys_path = '/content/drive/MyDrive/EMG-classification'
sys.path.append(sys_path)
from util import *
from dataloader import *

In [4]:
# DataLoader

record_data0501 = np.load('/content/drive/MyDrive/EMG-classification/dataset/rawdata/record_data0501.npy')
record_data0502 = np.load('/content/drive/MyDrive/EMG-classification/dataset/rawdata/record_data0502.npy')
record_data0503 = np.load('/content/drive/MyDrive/EMG-classification/dataset/rawdata/record_data0503.npy')
record_data0505 = np.load('/content/drive/MyDrive/EMG-classification/dataset/rawdata/record_data0505.npy')
record_data0506 = np.load('/content/drive/MyDrive/EMG-classification/dataset/rawdata/record_data0506.npy')
labels0501 = np.load('/content/drive/MyDrive/EMG-classification/dataset/label/label0501.npy')
labels0502 = np.load('/content/drive/MyDrive/EMG-classification/dataset/label/label0502.npy')
labels0503 = np.load('/content/drive/MyDrive/EMG-classification/dataset/label/label0503.npy')
labels0505 = np.load('/content/drive/MyDrive/EMG-classification/dataset/label/label0505.npy')
labels0506 = np.load('/content/drive/MyDrive/EMG-classification/dataset/label/label0506.npy')

signal0501 = record_data0501.flatten()
signal0502 = record_data0502.flatten()
signal0503 = record_data0503.flatten()
signal0505 = record_data0505.flatten()
signal0506 = record_data0506.flatten()

train_label = labels0502
train_record_data = record_data0502
train_signal = signal0502
val_record_data, val_signal, val_label = record_data0501, signal0501, labels0501

# train_label = np.concatenate((labels0502, labels0503, labels0505, labels0506), axis=0)
# train_record_data = np.concatenate((record_data0502, record_data0503, record_data0505, record_data0506), axis=0)
# train_signal = np.concatenate((signal0502, signal0503, signal0505, signal0506))
# val_record_data, val_signal, val_label = record_data0501, signal0501, labels0501
# assert train_label.shape[0] == train_record_data.shape[0]
# print("train data:{}\ntest data:{}\n".format(train_record_data.shape, record_data0501.shape))

# # val - 0502
# train_label = np.concatenate((labels0501, labels0503, labels0505, labels0506), axis=0)
# train_record_data = np.concatenate((record_data0501, record_data0503, record_data0505, record_data0506), axis=0)
# train_signal = np.concatenate((signal0501, signal0503, signal0505, signal0506))
# val_record_data, val_signal, val_label = record_data0502, signal0502, labels0502

# val - 0503
# train_label = np.concatenate((labels0501, labels0502, labels0505, labels0506), axis=0)
# train_record_data = np.concatenate((record_data0501, record_data0502, record_data0505, record_data0506), axis=0)
# train_signal = np.concatenate((signal0501, signal0502, signal0505, signal0506))
# val_record_data, val_signal, val_label = record_data0503, signal0503, labels0503

# val 05
# train_label = np.concatenate((labels0501, labels0502, labels0503, labels0506), axis=0)
# train_record_data = np.concatenate((record_data0501, record_data0502, record_data0503, record_data0506), axis=0)
# train_signal = np.concatenate((signal0501, signal0502, signal0503, signal0506))
# val_record_data, val_signal, val_label = record_data0505, signal0505, labels0505

# val06
# train_label = np.concatenate((labels0501, labels0502, labels0505, labels0503), axis=0)
# train_record_data = np.concatenate((record_data0501, record_data0502, record_data0505, record_data0503), axis=0)
# train_signal = np.concatenate((signal0501, signal0502, signal0505, signal0503))
# val_record_data, val_signal, val_label = record_data0506, signal0506, labels0506

In [ ]:
def reference(signal,length_of_win,fft_length,start,end):
    fs = 1200
    win = s.get_window('hann', length_of_win) # the number of samples in the window// win:ndarray[length_of_win]
    norm_signal = []
    for i in range(start, end):
        [f, t, stft_signal] = s.spectrogram(signal[i*1200+40:(i+1)*1200], fs,
                                    window=win,
                                    nperseg=length_of_win,
                                    noverlap=0.8*length_of_win,
                                    nfft = fft_length)
        stft_signal = abs(stft_signal)
        norm_signal.append(stft_signal)
    norm_signal = np.hstack(norm_signal)
    reference_mean_norm = 10*np.log10(np.mean(norm_signal,axis = 1))
    reference_std_norm = np.std(norm_signal,axis = 1)
    return reference_mean_norm

In [ ]:
length_of_win,fft_length,start,end = 100,256,0,100
reference_0501 = reference(signal0501,length_of_win,fft_length,start,end)
reference_0502 = reference(signal0502,length_of_win,fft_length,start,end)
reference_0503 = reference(signal0503,length_of_win,fft_length,start,end)
reference_0505 = reference(signal0505,length_of_win,fft_length,start,end)
reference_0506 = reference(signal0506,length_of_win,fft_length,start,end)

In [5]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int,float)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [6]:
class Netone(nn.Module):
    def __init__(self, num_classes, hidden_size, num_layers, middle_feature):
        super(Netone, self).__init__()
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = False
        self.middle_feature = middle_feature

        self.embedding = nn.Sequential(
            nn.Conv2d(1, 2, kernel_size=(3, 1), padding=(1, 0), stride=1),
            nn.ReLU(),
            nn.Conv2d(2, 4, kernel_size=(3, 1), padding=(1, 0), stride=1),
            nn.ReLU())

        # self.embedding_dim = 1281
        # self.embedding_dim = 129*4
        self.embedding_dim = 51*4
        # h0 = 256 num_layers = 1

        self.BiLSTM = nn.LSTM(input_size=self.embedding_dim,
                              hidden_size=self.hidden_size,
                              num_layers=self.num_layers,
                              dropout=0.2,
                              bidirectional=self.bidirectional)

        self.MLP = nn.Sequential(
            nn.Linear(self.hidden_size, self.middle_feature),
            nn.ReLU(),
            nn.Linear(self.middle_feature, num_classes)
        )  # remove the mlp and generate the irritation state at each time step

    def forward(self, datas):
        batch, f, t = datas.shape
        # print('input:', batch, f, t)# overall patient data = [1, 129, 54]
        # datas = datas.view(batch, 1, t, f)  # [N, C, H, W]
        datas = datas.reshape(batch, 1, t, f)
        embedded_datas = self.embedding(datas)
        # print('after conv:',embedded_datas.size()) # [1, 4, 56, 129]
        # embedded_datas = embedded_datas.permute(3, 0, 1, 2).view(f, batch, -1)
        embedded_datas = embedded_datas.permute(2, 0, 1, 3).reshape(t, batch, -1)
        # print('before lstm:',embedded_datas.size())
        out, _ = self.BiLSTM(embedded_datas)
        # print('after lstm:',out.size()) # [56, 1, 256]
        # out = out[-1, :, :]  # 20,512
        out = self.MLP(out.squeeze())
        # print('output shape:', out.shape)
        return out

In [ ]:
# FFT
testSignal0501 = signal0501[:1200]
# print(testSignal0501.shape)
window_length = 100
stride = 20
# Define a function to perform sliding FFT on the signal
def sliding_fft(signal, window_length, stride):
    # Calculate the number of windows
    num_windows = (len(signal) - window_length) // stride + 1
    print(num_windows)

    # Initialize an array to store the FFT spectra
    spectra = np.zeros((num_windows, window_length // 2 + 1))

    # Loop over the windows and calculate the FFT for each window
    for i in range(num_windows):
        start = i * stride
        end = start + window_length
        window = signal[start:end]
        window_spectrum = np.abs(fft(window)[:window_length // 2 + 1])
        spectra[i] = window_spectrum

    return spectra

# Process the signal with sliding FFT
spectra = sliding_fft(testSignal0501, window_length, stride)
print(spectra.shape)

56
(56, 51)


In [7]:
def signal_fft(signal, window_length, stride):
    # Calculate the number of windows
    num_windows = (len(signal) - window_length) // stride + 1

    # Initialize an array to store the FFT spectra
    spectra = np.zeros((num_windows, window_length // 2 + 1))

    # Loop over the windows and calculate the FFT for each window
    for i in range(num_windows):
        start = i * stride
        end = start + window_length
        window = signal[start:end]
        window_spectrum = np.abs(fft(window)[:window_length // 2 + 1])
        spectra[i] = window_spectrum

    return spectra

In [8]:
window_length = 100
stride = 20

train_raw = signal_fft(train_signal[:1200], window_length, stride)
num_windows = (len(train_signal) - window_length) // stride + 1
input_length, input_num = train_raw.shape # 56, 51
train_raw = train_raw.T.reshape(1,input_num,input_length) # [1, 51, 56]
train_label = np.repeat(train_label, 56)
# print(train_raw.shape)

eps = 1
#set hidden size
hidden_size = 256
n_class = 9
learning_rate = 0.01
max_epochs = 20
middle_features = 128
model = Netone(num_classes=n_class, hidden_size=hidden_size, num_layers=1, middle_feature=middle_features) #num_classes, hidden_size, num_layers, bidirectional, middle_feature
#model.cuda()
# criterion = nn.CrossEntropyLoss()
criterion = FocalLoss()
optimizer = torch.optim.Adam(model.parameters())

train_acc = []
val_acc = []

for epoch in range(max_epochs):
    correct_all = 0
    total_loss = 0
    train_length = 0
    predicted_train_csv = []
    real_train_csv = []

    for batch_idx in range(input_length, num_windows - input_length):
      # sliding fft
      start = batch_idx*stride
      window = train_signal[start:(start+window_length)]
      tail = np.abs(fft(window)[:window_length // 2 + 1]) # (51,)
      train_raw = np.concatenate((train_raw[:, :, 1:], tail.reshape(1, -1, 1)), axis=2)
      # train_raw = train_raw.unsqueeze().reshape(0,2,1)
      # print(train_raw.shape)
      train_raw = torch.from_numpy(train_raw)
      label_raw = train_label[batch_idx-input_length+1]
      batch_x = Variable(train_raw.float())
      optimizer.zero_grad()
      output = model(batch_x)
      _, predicted_train = torch.max(output.data, 1)
      if label_raw==predicted_train[0]: correct_all += 1
      train_length += 1

      predicted_train_csv.append(predicted_train[0].cpu().detach().numpy())
      real_train_csv.append(int(label_raw))

      label_raw_tensor = torch.tensor(label_raw, dtype=torch.long)
      loss = criterion(output, label_raw_tensor)#.float().reshape(-1, 1)
      total_loss += loss
      loss.backward()
      optimizer.step()

    print('\n************************************************************************************************')
    print("Epoch {}/{}:, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch+1,
        max_epochs,
        float(total_loss / train_length),
        float(optimizer.param_groups[0]['lr'])))
    print('train_acc:{}\n'.format(correct_all/train_length))

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
<ipython-input-8-cdcf9e11296b>:54: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  label_raw_tensor = torch.tensor(label_raw, dtype=torch.long)
<ipython-input-5-341bab4101f7>:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


IndexError: ignored

In [ ]:
# Use FFT
eps = 1
input_size = 129
#set hidden size
hidden_size = 256
n_class = 9
learning_rate = 0.1
max_epochs = 20
middle_features = 128
model = Netone(num_classes=n_class, hidden_size=hidden_size, num_layers=1, middle_feature=middle_features) #num_classes, hidden_size, num_layers, bidirectional, middle_feature
#model.cuda()
# criterion = nn.CrossEntropyLoss()
criterion = FocalLoss()
optimizer = torch.optim.Adam(model.parameters())

train_acc = []
val_acc = []

for epoch in range(max_epochs):
    correct_all = 0
    total_loss = 0
    train_length = 0
    predicted_train_csv = []
    real_train_csv = []

    for batch_idx in range(0,train_record_data.shape[0],eps):
        # tic = time.perf_counter()
        if batch_idx > train_record_data.shape[0]-eps: break
        train_eIMFs = train_signal[1200*batch_idx:1200*(batch_idx+eps)+1]
        train_raw = spectro(train_record_data[batch_idx:batch_idx+eps], train_eIMFs, 100, 256)
        train_raw = torch.from_numpy(train_raw)
        print(train_raw.shape)
        # train_filtered = train_raw[:, 5:, :]
        # print("shape after cutoff:{}\n".format(train_filtered.shape))
        label_raw = train_label[batch_idx:batch_idx+eps]
        # print("train_raw shape:{}\n label_raw shape:{}".format(train_raw.shape, label_raw.shape)) # (1, 129, 56)

        label = np.repeat(label_raw, train_raw.shape[-1])
        label = torch.from_numpy(label)
        # train_set = train_filtered.permute(0, 2, 1)
        # train_set = train_set.reshape(train_set.shape[0]*train_set.shape[1], -1)
        # print("train_set shape:{}\nlabel shape:{}\n".format(train_set.shape, label.shape)) # [310284, 129]

        # batch_x = torch.unsqueeze(train_raw, dim=0) # [1, 263736, 129]
        batch_x = Variable(train_raw.float())  # .cuda()
        batch_y = label.long()

        optimizer.zero_grad()
        output = model(batch_x)  # ,hidden = model(batch_x,None)

        # toc = time.perf_counter()
        # print(f"test epochs time in {toc - tic:0.4f} seconds")

        _, predicted_train = torch.max(output.data, 1)
        assert(len(predicted_train)==len(batch_y))
        correct_all += sum(batch_y.eq(predicted_train))
        train_length += len(batch_y)

        predicted_train_csv.append(predicted_train.cpu().detach().numpy())
        real_train_csv.append(batch_y.cpu().detach().numpy())

        loss = criterion(output, batch_y.squeeze())#.float().reshape(-1, 1)
        total_loss += loss
        loss.backward()
        optimizer.step()

    np.save('/content/drive/MyDrive/EMG-classification/logs/fft/val01_train_pred_'+ str(correct_all / train_length) + '.npy', predicted_train_csv)
    np.save('/content/drive/MyDrive/EMG-classification/logs/fft/val01_train_real_'+ str(correct_all / train_length) + '.npy', real_train_csv)

    print('\n************************************************************************************************')
    print("Epoch {}/{}:, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch+1,
        max_epochs,
        float(total_loss / train_length),
        float(optimizer.param_groups[0]['lr'])))
    print('train_acc:{}\n'.format(correct_all/train_length))

    if epoch%2==0:
        correct_all = 0
        predicted_val_csv = []
        real_val_csv = []
        val_length = 0
        for batch_idx in range(0,val_record_data.shape[0],eps):
            # tic = time.perf_counter()
            if batch_idx > val_record_data.shape[0]-eps: break
            # val_eIMFs = eemd.eemd(val_signal[1200*batch_idx:1200*(batch_idx+eps)+1])
            val_eIMFs = val_signal[1200*batch_idx:1200*(batch_idx+eps)+1]
            val_raw = spectro(val_record_data[batch_idx:batch_idx+eps], val_eIMFs, 100, 256)
            val_raw = torch.from_numpy(val_raw)
            # val_filtered = val_raw[:, 5:, :]
            label_raw = val_label[batch_idx:batch_idx+eps]
            # print("val_raw shape:{}\n label_raw shape:{}".format(val_raw.shape, label_raw.shape)) # (1, 129, 56)

            label = np.repeat(label_raw, val_raw.shape[-1])
            label = torch.from_numpy(label)
            # val_set = val_filtered.permute(0, 2, 1)
            # val_set = val_set.reshape(val_set.shape[0]*val_set.shape[1], -1)
            # print("val_set shape:{}\nlabel shape:{}\n".format(val_set.shape, label.shape)) # [310284, 129]

            # batch_x = torch.unsqueeze(val_raw, dim=0) # [1, 263736, 129]
            batch_x = Variable(val_raw.float())  # .cuda()
            batch_y = label.long()

            output = model(batch_x)  # ,hidden = model(batch_x,None)

            # toc = time.perf_counter()
            # print(f"Test time cost in {toc - tic:0.4f} seconds")

            _, predicted_val = torch.max(output.data, 1)
            # print("predicted_val shape:{}\nbatch_y shape:{}\n".format(predicted_val.shape, batch_y.shape))
            # if len(predicted_val)!=len(batch_y): 
            #   print("label shape:{}\nval_raw shape:{}\nlabel_raw shape:{}".format(label.shape, val_raw.shape, label_raw.shape))
            correct_all += sum(batch_y.eq(predicted_val))
            val_length += len(batch_y)
            predicted_val_csv.append(predicted_val.cpu().detach().numpy())
            real_val_csv.append(batch_y.cpu().detach().numpy())

        print('\n************************************************************************************************')
        print("Epoch {}/{}:\ntest_acc:{}\n".format(epoch+1, max_epochs, correct_all/val_length))

        np.save('/content/drive/MyDrive/EMG-classification/logs/fft/val01_val_pred_'+ str(correct_all / val_length) + '.npy', predicted_val_csv)
        np.save('/content/drive/MyDrive/EMG-classification/logs/fft/val01_val_real_'+ str(correct_all / val_length) + '.npy', real_val_csv)

In [ ]:
# input is added with window
eps = 1
input_size = 129
#set hidden size
hidden_size = 256
n_class = 9
learning_rate = 0.1
max_epochs = 20
middle_features = 128
model = Netone(num_classes=n_class, hidden_size=hidden_size, num_layers=1, middle_feature=middle_features) #num_classes, hidden_size, num_layers, bidirectional, middle_feature
#model.cuda()
# criterion = nn.CrossEntropyLoss()
criterion = FocalLoss()
optimizer = torch.optim.Adam(model.parameters())

train_acc = []
val_acc = []

for epoch in range(max_epochs):
    correct_all = 0
    total_loss = 0
    train_length = 0
    predicted_train_csv = []
    real_train_csv = []

    for batch_idx in range(0,train_record_data.shape[0],eps):
        # tic = time.perf_counter()
        if batch_idx > train_record_data.shape[0]-eps: break
        train_eIMFs = train_signal[1200*batch_idx:1200*(batch_idx+eps)+1]
        train_raw = spectro(train_record_data[batch_idx:batch_idx+eps], train_eIMFs, 100, 256)
        train_raw = torch.from_numpy(train_raw)
        # train_filtered = train_raw[:, 5:, :]
        # print("shape after cutoff:{}\n".format(train_filtered.shape))
        label_raw = train_label[batch_idx:batch_idx+eps]
        # print("train_raw shape:{}\n label_raw shape:{}".format(train_raw.shape, label_raw.shape)) # (1, 129, 56)

        label = np.repeat(label_raw, train_raw.shape[-1])
        label = torch.from_numpy(label)
        # train_set = train_filtered.permute(0, 2, 1)
        # train_set = train_set.reshape(train_set.shape[0]*train_set.shape[1], -1)
        # print("train_set shape:{}\nlabel shape:{}\n".format(train_set.shape, label.shape)) # [310284, 129]

        # batch_x = torch.unsqueeze(train_raw, dim=0) # [1, 263736, 129]
        batch_x = Variable(train_raw.float())  # .cuda()
        batch_y = label.long()

        optimizer.zero_grad()
        output = model(batch_x)  # ,hidden = model(batch_x,None)

        # toc = time.perf_counter()
        # print(f"test epochs time in {toc - tic:0.4f} seconds")

        _, predicted_train = torch.max(output.data, 1)
        assert(len(predicted_train)==len(batch_y))
        correct_all += sum(batch_y.eq(predicted_train))
        train_length += len(batch_y)

        predicted_train_csv.append(predicted_train.cpu().detach().numpy())
        real_train_csv.append(batch_y.cpu().detach().numpy())

        loss = criterion(output, batch_y.squeeze())#.float().reshape(-1, 1)
        total_loss += loss
        loss.backward()
        optimizer.step()

    np.save('/content/drive/MyDrive/EMG-classification/logs/cnnlstm/val06_train_pred_'+ str(correct_all / train_length) + '.npy', predicted_train_csv)
    np.save('/content/drive/MyDrive/EMG-classification/logs/cnnlstm/val06_train_real_'+ str(correct_all / train_length) + '.npy', real_train_csv)

    print('\n************************************************************************************************')
    print("Epoch {}/{}:, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch+1,
        max_epochs,
        float(total_loss / train_length),
        float(optimizer.param_groups[0]['lr'])))
    print('train_acc:{}\n'.format(correct_all/train_length))

    if epoch%2==0:
        correct_all = 0
        predicted_val_csv = []
        real_val_csv = []
        val_length = 0
        for batch_idx in range(0,val_record_data.shape[0],eps):
            # tic = time.perf_counter()
            if batch_idx > val_record_data.shape[0]-eps: break
            # val_eIMFs = eemd.eemd(val_signal[1200*batch_idx:1200*(batch_idx+eps)+1])
            val_eIMFs = val_signal[1200*batch_idx:1200*(batch_idx+eps)+1]
            val_raw = spectro(val_record_data[batch_idx:batch_idx+eps], val_eIMFs, 100, 256)
            val_raw = torch.from_numpy(val_raw)
            # val_filtered = val_raw[:, 5:, :]
            label_raw = val_label[batch_idx:batch_idx+eps]
            # print("val_raw shape:{}\n label_raw shape:{}".format(val_raw.shape, label_raw.shape)) # (1, 129, 56)

            label = np.repeat(label_raw, val_raw.shape[-1])
            label = torch.from_numpy(label)
            # val_set = val_filtered.permute(0, 2, 1)
            # val_set = val_set.reshape(val_set.shape[0]*val_set.shape[1], -1)
            # print("val_set shape:{}\nlabel shape:{}\n".format(val_set.shape, label.shape)) # [310284, 129]

            # batch_x = torch.unsqueeze(val_raw, dim=0) # [1, 263736, 129]
            batch_x = Variable(val_raw.float())  # .cuda()
            batch_y = label.long()

            output = model(batch_x)  # ,hidden = model(batch_x,None)

            # toc = time.perf_counter()
            # print(f"Test time cost in {toc - tic:0.4f} seconds")

            _, predicted_val = torch.max(output.data, 1)
            # print("predicted_val shape:{}\nbatch_y shape:{}\n".format(predicted_val.shape, batch_y.shape))
            # if len(predicted_val)!=len(batch_y): 
            #   print("label shape:{}\nval_raw shape:{}\nlabel_raw shape:{}".format(label.shape, val_raw.shape, label_raw.shape))
            correct_all += sum(batch_y.eq(predicted_val))
            val_length += len(batch_y)
            predicted_val_csv.append(predicted_val.cpu().detach().numpy())
            real_val_csv.append(batch_y.cpu().detach().numpy())

        print('\n************************************************************************************************')
        print("Epoch {}/{}:\ntest_acc:{}\n".format(epoch+1, max_epochs, correct_all/val_length))

        np.save('/content/drive/MyDrive/EMG-classification/logs/cnnlstm/val06_val_pred_'+ str(correct_all / val_length) + '.npy', predicted_val_csv)
        np.save('/content/drive/MyDrive/EMG-classification/logs/cnnlstm/val06_val_real_'+ str(correct_all / val_length) + '.npy', real_val_csv)

<ipython-input-5-341bab4101f7>:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)



************************************************************************************************
Epoch 1/20:, Train Loss 0.0071, Learning Rate 0.0010
train_acc:0.9093442559242249


************************************************************************************************
Epoch 1/20:
test_acc:0.6672337055206299


************************************************************************************************
Epoch 2/20:, Train Loss 0.0077, Learning Rate 0.0010
train_acc:0.9042321443557739


************************************************************************************************
Epoch 3/20:, Train Loss 0.0070, Learning Rate 0.0010
train_acc:0.9080585241317749


************************************************************************************************
Epoch 3/20:
test_acc:0.6672337055206299


************************************************************************************************
Epoch 4/20:, Train Loss 0.0077, Learning Rate 0.0010
train_acc:0.9004058241844177